In [ ]:
%pip install SimpleITK

In [ ]:
import os
from tqdm import tqdm
import random
import shutil

import SimpleITK as sitk
import matplotlib.pyplot as plt
import cv2
import shutil
from PIL import Image
import torch

import os
from glob import glob
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
def preprocess(ori_path: str, pro_path: str):
    id = 0
    for path in os.listdir(ori_path):
        if path.find('mhd') >= 0:
            id = int(path.split('.')[0])
            # save_content = pro_path
            # if os.path.exists(save_content):
            #     shutil.rmtree(save_content)
            # os.makedirs(save_content)
          

            data_mhd = sitk.ReadImage(os.path.join(ori_path, path))

            spacing = data_mhd.GetSpacing()
            img_arr = sitk.GetArrayFromImage(data_mhd)

            # mha_img = sitk.ReadImage('train/12399485.mhd')
            # img_arr = sitk.GetArrayFromImage(mha_img)# ndarray

            level = -200
            window = 1600

            window_minimum = level - window / 2
            window_maximum = level + window / 2
            img_arr[img_arr < window_minimum]= window_minimum
            img_arr[img_arr > window_maximum]= window_maximum

            # mha_img = sitk.GetImageFromArray(img_arr)
            # sitk.WriteImage(mha_img, 'train/12399485-pro.mhd')
            scan = img_arr

            for i in tqdm(range(len(scan))):
                img = cv2.normalize(scan[i], None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
                img = Image.fromarray(img)
                save_path = os.path.join(pro_path, f'{id}_{i}.png')
                img.save(save_path)

In [ ]:
#调用
low_dose_path = 'datasets/ct/final'  # 原数据集路径
low_dose_slice_path = 'datasets/ct/final_test'  # 保存路径


#read(low_dose_path, low_dose_slice_path)
preprocess(low_dose_path, low_dose_slice_path)

In [ ]:
test = os.listdir(low_dose_slice_path)
print(len(test))

In [ ]:
!python test.py --dataroot datasets/ct/testA --name 512 --model test --no_dropout --num_test 281 --preprocess none --load_size 512

In [ ]:
from tqdm import tqdm

def delete_real(leadpath):
    for file in tqdm(os.listdir(leadpath)):
        if file.find('real') >= 0:
            os.remove(os.path.join(leadpath, file))

leadpath = 'results/512/test_latest/images'
delete_real(leadpath)